In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from imblearn.under_sampling import RandomUnderSampler

## AJUSTANDO OS DADOS

In [ ]:
dados = pd.read_csv('dados.csv', sep = ';')
#TIRAR OS DADOS INCONCLUSIVOS
dados = dados[dados['test_result'] != 'Inconclusivo']
# Trocando as variáveis
for coluna in dados.columns:
    dados[coluna] = dados[coluna].replace('f', 0)
for coluna in dados.columns:
    dados[coluna] = dados[coluna].replace('t', 1)

dados.head()

,test_result,Gender,idade_anos,priority,CovidConfirmadoCasa,ProfissionalSaude,Febre,Tosse,DorGarganta,Coriza,...,Transplantado,GestacaoRisco,DoenteFigado,TratamentoImunossupressor,NA-Risco,Indicado,DataDosSintomas,SemanaEpidemiologica,DiasAteDiagnostico,Bairro
0,Negativo,Masculino,30,A,1,0,1,1,1,0,...,0,0,0,0,0,1,10/06/2020 20:10,7,4,Meier - Rio De Janeiro - RJ
1,Negativo,Feminino,24,A,0,1,0,1,1,1,...,0,0,0,0,0,0,10/06/2020 19:35,7,4,Inga - Niteroi - RJ
2,Negativo,Masculino,52,A,1,0,1,1,0,1,...,0,0,0,0,1,1,01/06/2020 00:00,6,35,Parque Prazeres - Campos Dos Goytacazes - RJ
3,Negativo,Feminino,37,A,1,1,0,1,1,0,...,0,0,0,0,1,1,28/06/2020 00:00,10,8,Centro - Duque De Caxias - RJ
4,Positivo,Masculino,66,A,1,0,0,1,0,1,...,0,0,0,0,1,1,10/06/2020 19:41,7,4,Barra Da Tijuca - Rio De Janeiro - RJ


In [ ]:
#Separando os dados
variaveis = ['test_result','Febre','Tosse', 'DorGarganta', 'Coriza', 'Mialgia', 'Enjoo', 'Diarreia', 'PerdaOlfato','FaltaAr']
dados_selecionados = dados[variaveis][:10000]

In [ ]:
dados_positivos = dados_selecionados[dados_selecionados['test_result'] == 'Positivo']
dados_positivos = dados_positivos.drop(columns = 'test_result')

dados_negativos = dados_selecionados[dados_selecionados['test_result'] == 'Negativo']
dados_negativos = dados_negativos.drop(columns = 'test_result')

dados_negativos.head()


,Febre,Tosse,DorGarganta,Coriza,Mialgia,Enjoo,Diarreia,PerdaOlfato,FaltaAr
0,1,1,1,0,1,1,1,1,1
1,0,1,1,1,1,0,0,0,0
2,1,1,0,1,0,0,0,1,0
3,0,1,1,0,1,0,0,0,0
5,0,0,0,0,0,0,0,1,0


##RESOLVENDO A PL

In [ ]:
matriz_p = np.array(dados_positivos)
matriz_n =np.array(dados_negativos)

# p = len(dados_negativos[0])
p = len(dados_negativos.columns)

n1 = len(dados_negativos)
n2 = len(dados_positivos)
n = n1+n2

print('Número de variáveis =', p )
print('Quantidade de dados negativos =', n1)
print('Quantidade de dados positivos =', n2)

Número de variáveis = 9
Quantidade de dados negativos = 8555
Quantidade de dados positivos = 1445


In [ ]:
#PESOS FUNÇÃO OBJETIVA
c1 = np.repeat(1/n1, n1)  #pesos para x= Negativo
c2 = np.repeat(1/n2, n2)  #pesos para x= Positivo
c3 = np.repeat(0, p + 1)  #pesos para a e b
c = np.concatenate((c1, c2, c3))
c = c.astype(float)

In [ ]:
#MATRIZ DE RESTRIÇÕES A
    #PRIMEIRO BLOCO
u1 = np.diag(np.repeat(-1, n1))
v1 = np.zeros((n1,n2))
ab1 = np.column_stack((-matriz_n, np.ones(n1)))
A1 = np.column_stack((u1,v1,ab1))
     #SEGUNDO BLOCO
u2 = u1
v2 = v1
ab2 = np.zeros((n1,p+1))
A2 = np.column_stack((u2,v2,ab2))
    #TERCEIRO BLOCO
u3 = np.zeros((n2,n1))
v3 = np.diag(np.repeat(-1, n2))
ab3 = np.column_stack((matriz_p, -np.ones(n2)))
A3 = np.column_stack((u3,v3,ab3))
     #QUARTO BLOCO
u4 = u3
v4 = v3
ab4 = np.zeros((n2,p+1))
A4 = np.column_stack((u4,v4,ab4))

A = np.concatenate((A1,A2,A3,A4))
A = A.astype(float)

In [ ]:
# VETORES B
B1 = np.repeat(-1,n1)
B2 = np.repeat(0,n1)
B3 = np.repeat(-1,n2)
B4 = np.repeat(0,n2)

B = np.concatenate((B1,B2,B3,B4))
B = B.astype(float)

In [ ]:
# SOLUÇÃO DA PL
nvar = n+p+1
resultado = linprog(c = c, A_ub = A, b_ub = B, bounds = (None,None), method='highs')
coeficientes = resultado.x[n:nvar]

In [ ]:
a = resultado.x[n:-1]
b = resultado.x[-1]
print(a,b)

[-0. -0. -0. -0. -0. -0. -0. -2. -0.] -1.0


In [ ]:
res_p = np.dot(matriz_p,a) - b +1
n_erro_p = sum(res_p > 0)
print('Quantidades de erros positivos:', n_erro_p)

Quantidades de erros positivos: 626
25% dos erros: 156.5


In [ ]:
res_n = np.dot(-matriz_n,a) + b +1
n_erro_n = sum(res_n > 0)
print('Quantidades de erros negativos:', n_erro_n)

Quantidades de erros negativos: 2082
25% dos erro: 520.5


In [ ]:
#MATRIZ DE CONFUSÃO
confusao = np.array([[ (n1 - n_erro_n), n_erro_n ],
                     [ n_erro_p, (n2 - n_erro_p)]])
confusao

array([[6473, 2082],
       [ 626,  819]])

## COMPARANDO COM O SVM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

SEED = 411
np.random.seed(SEED)

sintomas = dados_selecionados.drop(columns = 'test_result')
diagnostico = dados_selecionados['test_result']

treino_x_c, teste_x_c, treino_y_c, teste_y_c = train_test_split(sintomas, diagnostico, test_size = 0.25,
                                                               stratify = diagnostico)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x_c), len(teste_x_c)))

classificador = LinearSVC(class_weight='balanced')
classificador.fit(treino_x_c, treino_y_c)
previsoes_c = classificador.predict(teste_x_c)
hiperplano = classificador.coef_

print('Resultado da classificação %.2f%%' %(classificador.score(teste_x_c,teste_y_c)*100))
y_true_c = teste_y_c
y_pred_c = previsoes_c
target_names = ['Negativo', 'Positivo']


resultado_c = classification_report(y_true_c, y_pred_c, target_names=target_names)
print(resultado_c)
print("Coeficientes do hiperplano separador:", hiperplano)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
Resultado da classificação 71.92%
              precision    recall  f1-score   support

    Negativo       0.91      0.74      0.82      2139
    Positivo       0.28      0.58      0.37       361

    accuracy                           0.72      2500
   macro avg       0.59      0.66      0.60      2500
weighted avg       0.82      0.72      0.75      2500

Coeficientes do hiperplano separador: [[ 0.18820414  0.07905083 -0.16617579 -0.02564468  0.13934198  0.00905053
   0.03305657  0.56302321  0.03914004]]


In [ ]:
confusao_c = confusion_matrix(teste_y_c, classificador.predict(teste_x_c))
confusao_c

array([[1589,  550],
       [ 152,  209]])

In [ ]:
hiperplano = classificador.coef_
hiperplano

array([[ 0.18820414,  0.07905083, -0.16617579, -0.02564468,  0.13934198,
         0.00905053,  0.03305657,  0.56302321,  0.03914004]])